<a href="https://colab.research.google.com/github/gabgovar/RNP_PyTorch/blob/main/Pre_processamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Créditos

O conteúdo deste notebook usa material das seguintes fontes:

- [Deep Learning Wizard](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/pytorch_logistic_regression/)

- [Deep Learning with PyTorch: Zero to GANs](https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans)

- [Machine Learning Glossary](https://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html)

# Regressão Linear

Nesta seção, vamos resolver um problema de regressão linear tentando usar classes fornecidas pelo PyTorch para manipulação de conjuntos de dados. Vamos perceber que o PyTorch fornece a implementação de diversas partes do *pipeline* de treinamento de uma rede neural artificial.

## Armazenamento de dados em tensores

Uma operação bastante comum é fazer a carga de dados para tensores para processamento posterior.

Para ilustrar essa operação, vamos usar um conjunto de dados (*dataset*) bem simples denominado [Linnerud](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_linnerud.html). Esse conjunto de dados tem apenas 20 exemplos. Cada exemplo é representado por 3 variáveis ​​independentes e 3 alvos. A descrição do conjunto de dados Linnerud é a seguinte: 

> “O conjunto de dados Linnerud pode ser usado para ajustar modelos de regressão de múltiplas saídas. É composto por três variáveis sobre atividade física (matriz de dados, $X$) e três variáveis que medem características ​​fisiológicas (matriz alvo, $y$) coletados de vinte homens de meia-idade em um clube de fitness.

- variáveis relacionadas ao perfil de atividade física do indivíduo ($X$)- :  "puxar ferro" (Chins), abdominais (Situps) e saltos (Jumps).

- variáveis relacionadas ao perfil fisiológico do indivíduo ($y$) - Peso (Weight), Cintura (Weist) e Pulso (Pulse).

Mais informações sobre esse conjunto de dados podem ser encontradas nos links abaixo:

- https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_linnerud.html

- https://scikit-learn.org/stable/datasets/toy_dataset.html#linnerrud-dataset

- https://ai.plainenglish.io/an-exploration-into-sklearns-linnerrud-multioutput-dataset-4e0ad110c728

Dimensões da matriz de dados e da matriz alvo:
- $X \in \mathbb{R}^{20 \times 3}$: matriz de dados (*data matrix*)
- $y \in \mathbb{R}^{20 \times 3}$: matriz alvo (*target matrix*)

In [1]:
from sklearn.datasets import load_linnerud
import torch

X_, y_ = load_linnerud(return_X_y = True)

X = torch.from_numpy(X_) # utlizando o numpy para compactando em tensores
y = torch.from_numpy(y_)

X, y = X.float(), y.float()

In [2]:
X.shape

torch.Size([20, 3])

## TensorDataset e DataLoader

Criaremos um objeto `TensorDataset`, que propicia acesso aos exemplos de um conjunto de dados representados como tensores. O objeto `TensorDataset` fornece uma API padrão para trabalhar com muitos tipos diferentes de conjuntos de dados em PyTorch.

In [3]:
from torch.utils.data import TensorDataset

In [4]:
# Define dataset
train_ds = TensorDataset(X, y) #particionando os dados X e y
train_ds[0:5] # insepcionando o objeto 

(tensor([[  5., 162.,  60.],
         [  2., 110.,  60.],
         [ 12., 101., 101.],
         [ 12., 105.,  37.],
         [ 13., 155.,  58.]]), tensor([[191.,  36.,  50.],
         [189.,  37.,  52.],
         [193.,  38.,  58.],
         [162.,  35.,  62.],
         [189.,  35.,  46.]]))

O `TensorDataset` permite fazer acesso a uma seção dos dados de treinamento usando a notação de indexação de array (`[0:5]` no código acima). Quando usamos essa indexação, obtemos como resultado uma tupla com dois elementos. O primeiro elemento contém as variáveis independentes para as linhas selecionadas e o segundo contém o alvos correspondentes.

Também criaremos um objeto `DataLoader`, que pode dividir os exemplos em um conjunto de dados em vários **lotes** (*batchs*) de um tamanho predefinido. Esse objeto também fornece outros funções utilitárias, como embaralhamento e amostragem aleatória dos dados.

In [5]:
from torch.utils.data import DataLoader

In [6]:
# Define o objeto DataLoader
batch_size = 6 # definindo o tamanho lotes do dataset
train_dl = DataLoader(train_ds, batch_size, shuffle=True) # é uma boa pratica fazer o shuffle dos dados

É comum usar um objeto `DataLoader` em um laço de repetição para percorrer os exemplos de um conjunto de dados. Em cada iteração do laço de repetição, um lote (*batch*) de exemplos é produzido. O tamanho do lote é definido na criação do objeto `DataLoader`. Veja o código abaixo.

In [ ]:
for i, (xb, yb) in enumerate(train_dl):
    print('Batch #%d:' % i)
    print(xb)
    print(yb)

Em cada iteração, o objeto `DataLoader` retorna um **lote** de dados com o tamanho de lote previamente fornecido (argumento `batch_size`). Se `shuffle` tiver sido definido como `True`, esse objeto também embaralha os dados do conjunto antes de criar lotes. 

O embaralhamento dos exemplos ajuda no processo de convergência do algoritmo de otimização, levando a uma redução mais rápida da função de custo. Veja essa [discussão](https://datascience.stackexchange.com/questions/24511/why-should-the-data-be-shuffled-for-machine-learning-tasks) para mais detalhes sobre isso.

## Construção do modelo

Vamos construir um modelo de rede neural bem simples, com apenas a camada de entrada (correspondente aos próprios dados de treinamento) e a camada de saída (i.e., sem camadas intermediárias).

Em vez de implementar a parte de código para inicializar os pesos e viéses, podemos definir o modelo usando a classe `nn.Linear` do PyTorch, que faz isso automaticamente.

In [8]:
# Define o modelo
model = torch.nn.Linear(3, 3) #Encapsulamento da função de ativação
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2920,  0.0425, -0.0058],
        [-0.1422, -0.4074,  0.3152],
        [ 0.5000, -0.1893,  0.5334]], requires_grad=True)
Parameter containing:
tensor([ 0.4418,  0.5371, -0.4734], requires_grad=True)


Observe que o objeto `nn.Linear` encapsula os parâmetros do modelo (i.e., a matriz de pesos $W$ e o vetor de viéses $b$). Esse mesmo objeto já implementa a operação de multiplicação de dos pesos pelos exemplos de entrada e a consecutiva adição do viés (*bias*). De fato, dada uma matriz de dados $X$ (com cada exemplo armazenado em uma de suas linhas), esse objeto Linear computa a expressão matemática abaixo:

$$
W \times X^T + b
$$

Veja o código abaixo.

In [9]:
print(X)
print(model(X))

tensor([[  5., 162.,  60.],
        [  2., 110.,  60.],
        [ 12., 101., 101.],
        [ 12., 105.,  37.],
        [ 13., 155.,  58.],
        [  4., 101.,  42.],
        [  8., 101.,  38.],
        [  6., 125.,  40.],
        [ 15., 200.,  40.],
        [ 17., 251., 250.],
        [ 17., 120.,  38.],
        [ 13., 210., 115.],
        [ 14., 215., 105.],
        [  1.,  50.,  50.],
        [  6.,  70.,  31.],
        [ 12., 210., 120.],
        [  4.,  60.,  25.],
        [ 11., 230.,  80.],
        [ 15., 225.,  73.],
        [  2., 110.,  43.]])
tensor([[  5.5171, -47.2586,   3.3677],
        [  4.1839, -25.6479,  11.7104],
        [  0.6435, -10.4805,  40.2838],
        [  1.1841, -32.2827,   5.3881],
        [  2.8949, -46.1751,   7.6257],
        [  3.3216, -27.9394,   4.8124],
        [  2.1766, -29.7691,   4.6787],
        [  3.7688, -38.6315,   0.2028],
        [  4.3270, -70.4654,  -9.4935],
        [  4.6934, -25.3349,  93.8701],
        [  0.3554, -38.7894,   5.5822],

Ao analisar o resultado acima, você pode ver uma grande diferença entre as previsões do modelo e os valores verdadeiros porque inicializamos nosso modelo com pesos e vieses aleatórios. Obviamente, não podemos esperar que um modelo cujos parâmetros foram iniciados aleatoriamente funcione adequadamente.

## Função de custo

Para que possamos melhorar nosso modelo, precisamos de uma forma objetiva de avaliar o desempenho preditivo dele. Podemos comparar as previsões do modelo com os alvos reais usando o seguinte método:

- Calcular a diferença entre as duas matrizes (`y_pred` e `y`).
- Elevar ao quadrado cada elemento da matriz de diferença para remover valores negativos.
- Calcular a média dos elementos na matriz resultante.

O resultado é um único número, conhecido como erro quadrático médio (MSE, *mean squared error*). Matematicamente, os passos acima se traduzem nas seguintes expressões, onde $n$ é a quantidade de variáveis alvo:

\begin{align}
D &= (y - y_{\text{pred}}) \\
S &= D \odot D \\
\operatorname{MSE} &= \frac{1}{n} \sum_{1 \leq i,j \leq n} S_{ij} 
\end{align}

O pacote `nn.Functional` fornece a implementação de muitas funções de custo úteis e vários outros utilitários. Em particular, vamos usar a implementação da função de custo MSE fornecida por esse pacote.

In [10]:
import torch.nn.functional as F

# Define loss function
loss_fn = F.mse_loss

Vamos computar o valor inicial da função de custo:

In [11]:
loss = loss_fn(model(X), y)
print(loss)

tensor(13078.7627, grad_fn=<MseLossBackward0>)


In [12]:
import math
math.sqrt(loss)

114.36241819458218

Podemos interpretar o valor produzido acima da seguinte forma: 

> Em média, cada previsão feita pelo modelo difere do alvo (valor verdadeiro) pela raiz quadrada do valor da função de custo. 

Objetivamente, esse resultado é muito ruim, considerando que os números que estamos tentando prever estão na faixa de 30-250. O resultado é chamado de perda porque indica o quão ruim o modelo é em prever as variáveis alvo. Representa a perda de informações no modelo: quanto menor a perda, melhor é o modelo.

## Cálculo dos gradientes

Com PyTorch, podemos calcular automaticamente os gradientes (i.e., as derivadas parciais da função de custo com relação a cada um dos parâmetros do modelo). Isso porque eles foram definidos com `requires_grad` igual a `True`. Esse cálculo é realizado ao invocar a função `backward`.

In [13]:
# Computa os gradientes (diferenciação automática; autograd)
loss.backward()

Os gradientes são armazenados na propriedade `.grad` dos respectivos tensores. Observe que as derivadas parciais da função de custo com relação a cada elemento da matriz de pesos $W$ podem ser organizados em outra matriz com as mesmas dimensões. O mesmo pode ser feito com relação ao vetor $b$. Se denotarmos por $J$ a função de custo e por $dW$ e $db$ essas novas matrizes, temos:

$$
dW = 
\begin{bmatrix}
\frac{\partial J}{\partial w_{11}} & \frac{\partial J}{\partial w_{12}} & \frac{\partial J}{\partial w_{13}}\\
\frac{\partial J}{\partial w_{21}} & \frac{\partial J}{\partial w_{22}} & \frac{\partial J}{\partial w_{23}}\\
\frac{\partial J}{\partial w_{31}} & \frac{\partial J}{\partial w_{32}} & \frac{\partial J}{\partial w_{33}}
\end{bmatrix}
$$

$$
db = 
\begin{bmatrix}
\frac{\partial J}{\partial b_{1}} \\
\frac{\partial J}{\partial b_{2}} \\
\frac{\partial J}{\partial b_{3}}
\end{bmatrix}
$$

Os matemáticos chamam cada uma das matrizes acima de [Jacobiana](https://en.wikipedia.org/wiki/Jacobian_matrix_and_determinant).

A função de custo MSE é uma função quadrática dos pesos e vieses, e nosso objetivo é encontrar o conjunto de parâmetros onde a função de custo é mínima. Se traçarmos um gráfico da função de custo com qualquer parâmetro individual (peso ou viés), ele se parecerá com a figura mostrada no link a seguir: https://www.geogebra.org/m/j8jqxyrs.

Um informação importante é que cada derivada parcial contida na Jacobiana $dW$ indica a taxa de variação da função de custo em uma direção específica, ou seja, a inclinação dessa função.

Se um elemento de $dW$ for **positivo**, então:
- **aumentar** ligeiramente o valor do parâmetro correspondente **aumenta** o valor da função de custo;
- **diminuir** ligeiramente o valor do peso correspondente  **diminui** o valor da função de custo.

Se um elemento de $dW$ for **negativo**, então:
- **aumentar** ligeiramente o valor do peso correspondente **diminui** o valor da função de custo;
- **diminuir** ligeiramente o valor do peso correspondente **aumenta** o valor da função de custo.

O aumento ou diminuição na função $J$ causado pela mudança no valor um elemento em $W$ é proporcional a esse valor. Essa observação forma a base do algoritmo de otimização de **gradiente descendente** que usaremos para melhorar nosso modelo.

> Podemos subtrair de cada elemento de $W$ uma pequena quantidade proporcional à derivada de $J$ com relação a esse elemento para reduzir ligeiramente o custo.

Por exemplo, para atualizar o peso $w_{11}$, a seguinte expressão deve ser usada:

$$
w_{11} \leftarrow w_{11} - \alpha \times \frac{\partial J}{\partial w_{11}}
$$

A mesma explicação dada acima no contexto da matriz de pesos ($W$) pode ser dada no contexto do vetor de viéses ($b$).

## Otimizador

Vamos usar o otimizador `optim.SGD`. SGD é a abreviatura de *sthocastic gradient descent* (descida do gradiente estocástica). O termo *estocástico* indica que as amostras são selecionadas em lotes aleatórios, em vez de como um único grupo.

In [15]:
# Define o otimizadora ser usado (Sthocastic Gradient Descent)
opt = torch.optim.SGD(model.parameters(), lr=1e-5) # lr = learn rate, o alfa da equação acima

Observe que `model.parameters()` é passado como um argumento para `optim.SGD` para que o otimizador saiba quais matrizes devem ser modificadas durante a etapa de atualização. Além disso, podemos especificar uma taxa de aprendizado que controla a quantidade pela qual os parâmetros são modificados.

A função `model.parameters()` encapsula uma estrutura de dados que armazena todos os tensores (matrizes ou vetores) de parâmetros do modelo. Veja o código a seguir.

In [16]:
for i in model.parameters():
  print(i)

Parameter containing:
tensor([[-0.2920,  0.0425, -0.0058],
        [-0.1422, -0.4074,  0.3152],
        [ 0.5000, -0.1893,  0.5334]], requires_grad=True)
Parameter containing:
tensor([ 0.4418,  0.5371, -0.4734], requires_grad=True)


## Treinamento do modelo

Agora estamos prontos para treinar o modelo. Seguiremos o mesmo processo para implementar a descida do gradiente:

1. Gerar previsões passando para o modelo alguns exemplos de treinamento;
2. Com as previsões geradas, calcular o valor da função de custo;
3. Calcular gradientes (derivadas parciais) com relação aos parâmetros (pesos e viéses);
4. Ajustar os parâmetros subtraindo deles uma pequena quantidade proporcional ao gradiente;
5. Redefinir os gradientes para zero.

Na terminologia de algoritmos de aprendizado de máquina baseados em gradiente, toda vez que o algoritmo executa os passos acima, dizemos que ele executou uma **iteração de treinamento**, ou simplesmente **iteração**.

Na implementação abaixo, passamos *lotes de dados* para o modelo (veja o passo 1 acima), em vez de passar todos os exemplos de treinamento de uma única vez. Vamos definir uma função auxiliar denominada `fit` que treina o modelo por um determinado número de **épocas**. Cada época corresponde a várias iterações.

In [17]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb, yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Algumas coisas a serem observadas no código acima:

- Usamos o objeto `DataLoader` definido anteriormente para obter lotes de dados para cada iteração.

- Em vez de implementar a atualização dos parâmetros (pesos e viéses), usamos a função `opt.step` para realizar essa atualização e [opt.zero_grad](https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html) para redefinir os gradientes para zero.

- Também adicionamos um trecho de código para registrar o valor da função de custo (computada em cada lote de dados) para cada 10ª época. Esse trecho nos permitirá rastrear o progresso do treinamento. `loss.item` retorna o valor real armazenado no tensor zero-dimensional da função de custo.

Vamos treinar o modelo por 100 épocas.

In [22]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 2002.7650
Epoch [20/100], Loss: 2403.7451
Epoch [30/100], Loss: 512.5737
Epoch [40/100], Loss: 4071.0598
Epoch [50/100], Loss: 1181.4362
Epoch [60/100], Loss: 2388.3684
Epoch [70/100], Loss: 7191.7720
Epoch [80/100], Loss: 411.2676
Epoch [90/100], Loss: 1843.1056
Epoch [100/100], Loss: 2739.9717


Vamos agora gerar previsões usando o modelo atual e verificar se elas estão próximas dos alvos.

In [26]:
y_pred = model(X)
y_pred

tensor([[171.0631,  34.2474,  55.3488],
        [114.6619,  23.3948,  35.4771],
        [107.9401,  21.9512,  34.8847],
        [116.3070,  22.8968,  40.6180],
        [168.1547,  33.2001,  57.2985],
        [107.4335,  21.6736,  34.6008],
        [109.8608,  21.8763,  37.0081],
        [133.9281,  26.7163,  44.2364],
        [217.8085,  42.5971,  75.3940],
        [258.9863,  52.7946,  80.1792],
        [134.7523,  26.2271,  48.4914],
        [222.3912,  44.4154,  72.7671],
        [228.8489,  45.5127,  75.7342],
        [ 51.5665,  11.0116,  14.5393],
        [ 76.5767,  15.4094,  25.5219],
        [221.5248,  44.3571,  71.8947],
        [ 65.3484,  13.2644,  21.3459],
        [244.6461,  48.5025,  81.0921],
        [242.0013,  47.6814,  81.9457],
        [115.7651,  23.4272,  36.6267]], grad_fn=<AddmmBackward0>)

Compare o tensor acima com o tensor de alvo exibido abaixo.

In [24]:
y

tensor([[191.,  36.,  50.],
        [189.,  37.,  52.],
        [193.,  38.,  58.],
        [162.,  35.,  62.],
        [189.,  35.,  46.],
        [182.,  36.,  56.],
        [211.,  38.,  56.],
        [167.,  34.,  60.],
        [176.,  31.,  74.],
        [154.,  33.,  56.],
        [169.,  34.,  50.],
        [166.,  33.,  52.],
        [154.,  34.,  64.],
        [247.,  46.,  50.],
        [193.,  36.,  46.],
        [202.,  37.,  62.],
        [176.,  37.,  54.],
        [157.,  32.,  52.],
        [156.,  33.,  54.],
        [138.,  33.,  68.]])

De fato, as previsões estão bem próximas dos alvos. Treinamos um modelo razoavelmente bom para prever o perfil fisiológico de um indivíduo, dado que sabemos seu perfil de realização de exercícios físicos.

O código abaixo exemplifica de que forma o modelo pode ser usado para fazer predições sobre novos indivíduos.

In [25]:
model(torch.tensor([[6., 152., 59.]])) # inferência

tensor([[161.1362,  32.2392,  52.4374]], grad_fn=<AddmmBackward0>)